# Téléchargement des données des prélèvements en eau avec Hub'Eau
*Antonio Andrade, ingénieur de données, [Office français de la biodiversité](https://ofb.gouv.fr/)*  

[Hub'Eau](https://hubeau.eaufrance.fr/) est une plateforme nationale d'API REST permettant d'automatiser l'accès et le traitement des données ouvertes du [Sytème d'Information sur l'Eau (SIE)](https://www.eaufrance.fr/le-systeme-dinformation-sur-leau-sie) Ce tutoriel illustre l'utilisation de l'[API Prélèvements en eau](https://hubeau.eaufrance.fr/page/api-prelevements-eau) pour télécharger les données déclaratives collectées par les agences et offices de l'eau et réunies dans la banque nationale des prélèvements en eau, [BNPE](https://bnpe.eaufrance.fr/).

## Chargement des modules
Commençons par charger en mémoire les modules python nécessaires pour nos traitements :
- les modules standards `os` et `requests` pour interagir avec le système d'exploitation et les API REST Hub'Eau
- le module `pandas` pour traiter des données structurées alphanumériques

In [1]:
# Chargement des modules nécessaires à nos traitements
import os
import requests

import pandas

print("Chargement OK")

Chargement OK


## Paramétrage du téléchargement
Poursuivons avec le paramétrage de notre traitement :

In [2]:
# URL de base des API Hub'Eau
base = "http://hubeau.eaufrance.fr/api"
# Point d'accès de l'API Hub'Eau Prélèvements en eau
endpoint = "v1/prelevements"
# Opération correspondant aux chroniques de prélèvement en eau
operation = "chroniques"
# Nombre de résultats par page
page_size = 1000
# Nombre maximum de résultats pour une recherche
max_results = 20000

# Années des prélèvements
years = ["2017", "2018", "2019"]
# Données attendues dans la réponse (cf. données disponibles du modèle Chonique)
fields = [
    "code_ouvrage",
    "nom_ouvrage",
    "code_commune_insee",
    "nom_commune",
    "code_departement",
    "libelle_departement"
    "code_usage",
    "libelle_usage",
    "volume",
    "code_qualification_volume",
    "libelle_qualification_volume",
    "producteur_donnee"
]

print("Paramétrage OK")

Paramétrage OK


## Téléchargement des données au format json
Construisons d'abord l'URL de téléchargement des chroniques recherchées :

In [3]:
# Paramètres de la recherche des chroniques
parameters = [
    {
        "parameter": "annee",
        "value": ",".join(years)
    },
    {
        "parameter": "fields",
        "value": ",".join(fields)
    },
    {
        "parameter": "size",
        "value": page_size
    }
]
# URL de téléchargement des résultats de la recherche
parameters = "&".join([f"{param['parameter']}={param['value']}" for param in parameters])
hubeau_url = f"{base}/{endpoint}/{operation}?{parameters}"
hubeau_url

'http://hubeau.eaufrance.fr/api/v1/prelevements/chroniques?annee=2017,2018,2019&fields=code_ouvrage,nom_ouvrage,code_commune_insee,nom_commune,code_departement,libelle_departementcode_usage,libelle_usage,volume,code_qualification_volume,libelle_qualification_volume,producteur_donnee&size=1000'

Téléchargeons maintenant les chroniques recherchées en commençant par la 1ère page :

In [4]:
# Récupération de la 1ère page de résultats
response = requests.get(hubeau_url)
# Conversion de la chaîne de texte au format json dans un dictionnaire python
response = response.json()

if response['count'] > max_results:
    print("Le nombre de résultats dépasse le seuil maximum de {0}. " \
          "Seuls les {0} premiers résultats seront téléchargés".format(max_results))

# Extraction des données de la 1ère page de résultats
data = response['data']
# Initialisation du nombre de résultats requêtés
nb_results = page_size

# Affichage du 1er résultat de recherche
data[0]

Le nombre de résultats dépasse le seuil maximum de 20000. Seuls les 20000 premiers résultats seront téléchargés


{'code_ouvrage': 'OPR0000000003',
 'volume': 11490.0,
 'libelle_usage': 'EAU POTABLE',
 'code_qualification_volume': '1',
 'libelle_qualification_volume': 'Correcte',
 'producteur_donnee': 'AERM',
 'code_commune_insee': '52332',
 'nom_commune': 'Val-de-Meuse',
 'code_departement': '52',
 'nom_ouvrage': "COMMUNE D'AVRECOURT"}

Téléchargeons ensuite les résultats page par page :

In [5]:
# Initialisation d'un indicateur de traitement de la dernière page
last_page = False

# Récupération des résultats des pages suivantes
# L'attribut next est null (None en Python) lorsque la dernière page est atteinte
while response['next'] is not None and not last_page:
    # URL de la ième page de résultats
    hubeau_url = response['next']

    if (nb_results + page_size) >= max_results:
        # Re paramétrage du nombre de résultats de la dernière page
        hubeau_url = hubeau_url.replace(
            f"size={page_size}", "size={}".format(max_results - nb_results)
        )
        last_page = True

    # Récupération de la ième page de résultats
    print(hubeau_url)
    response = requests.get(hubeau_url)
    # Conversion de la chaîne de texte au format json dans un dictionnaire python
    response = response.json()
    # Extraction et union des données d'analyse
    data += response['data']
    # Incrémentation du nombre de résultats requêtés
    nb_results += page_size

# Affichage du nombre de résultats récupérés
print("Nb de résultats récupérés : {}".format(len(data)))

https://hubeau.eaufrance.fr/api/v1/prelevements/chroniques?annee=2017,2018,2019&fields=code_ouvrage,nom_ouvrage,code_commune_insee,nom_commune,code_departement,libelle_departementcode_usage,libelle_usage,volume,code_qualification_volume,libelle_qualification_volume,producteur_donnee&page=2&size=1000
https://hubeau.eaufrance.fr/api/v1/prelevements/chroniques?annee=2017,2018,2019&fields=code_ouvrage,nom_ouvrage,code_commune_insee,nom_commune,code_departement,libelle_departementcode_usage,libelle_usage,volume,code_qualification_volume,libelle_qualification_volume,producteur_donnee&page=3&size=1000
https://hubeau.eaufrance.fr/api/v1/prelevements/chroniques?annee=2017,2018,2019&fields=code_ouvrage,nom_ouvrage,code_commune_insee,nom_commune,code_departement,libelle_departementcode_usage,libelle_usage,volume,code_qualification_volume,libelle_qualification_volume,producteur_donnee&page=4&size=1000
https://hubeau.eaufrance.fr/api/v1/prelevements/chroniques?annee=2017,2018,2019&fields=code_ouvra

## Enregistrement des données au format csv
Terminons notre traitement avec l'enregistrement des données au format CSV :

In [6]:
# Création d'un DataFrame à partir du dictionnaire python
df = pandas.DataFrame(data)

# Enregistrement des données dans un fichier csv stocké dans le même dossier que
# le notebook
df.to_csv(
    os.path.join(os.getcwd(), "bnpe_chroniques_{}.csv".format("-".join(years))),
    sep=';',
    header=True,
    index=False,
    mode='w',
    encoding='utf8',
    chunksize=100000)

# Affichage des 1ères lignes de données du DataFrame
df.head()

,code_ouvrage,volume,libelle_usage,code_qualification_volume,libelle_qualification_volume,producteur_donnee,code_commune_insee,nom_commune,code_departement,nom_ouvrage
0,OPR0000000003,11490.0,EAU POTABLE,1,Correcte,AERM,52332,Val-de-Meuse,52,COMMUNE D'AVRECOURT
1,OPR0000000003,10290.0,EAU POTABLE,1,Correcte,AERM,52332,Val-de-Meuse,52,COMMUNE D'AVRECOURT
2,OPR0000000003,9316.0,EAU POTABLE,1,Correcte,AERM,52332,Val-de-Meuse,52,COMMUNE D'AVRECOURT
3,OPR0000000004,740.0,EAU POTABLE,1,Correcte,AERM,52064,Bourmont-entre-Meuse-et-Mouzon,52,COMMUNE DE BOURMONT-ENTRE-MEUSE-ET-MOUZON
4,OPR0000000004,12488.0,EAU POTABLE,1,Correcte,AERM,52064,Bourmont-entre-Meuse-et-Mouzon,52,COMMUNE DE BOURMONT-ENTRE-MEUSE-ET-MOUZON


Merci pour votre attention

Références :
- [Hub'Eau](https://hubeau.eaufrance.fr/)
- [BNPE](https://bnpe.eaufrance.fr/)
- [pandas](https://pandas.pydata.org/)